In [2]:
import numpy as np 
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv('/Users/sahil_jangid/course/python/project#1/saas_user_activity.csv')

In [4]:
data.head()

,event_id,user_id,event_date,country,subscription_plan,device_type,traffic_source,event_type,session_duration_mins,monthly_revenue,customer_support_tickets
0,E50000,U10969,2024-01-18,UK,Free,Tablet,Referral,Login,37,$56.27,2
1,E50001,U10772,2024-02-27,Australia,Basic,Mobile,Email,Feature_Use,142,$81.08,1
2,E50002,U10048,2023-08-28,Australia,Free,Mobile,Ads,Feature_Use,176,$304.46,4
3,E50003,U10368,2023-03-01,India,Basic,Desktop,Referral,Login,1,$259.81,1
4,E50004,U11144,2023-08-10,Germany,Basic,Mobile,Organic,Login,39,$119.56,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   event_id                  5000 non-null   object
 1   user_id                   5000 non-null   object
 2   event_date                5000 non-null   object
 3   country                   5000 non-null   object
 4   subscription_plan         5000 non-null   object
 5   device_type               5000 non-null   object
 6   traffic_source            5000 non-null   object
 7   event_type                5000 non-null   object
 8   session_duration_mins     5000 non-null   int64 
 9   monthly_revenue           5000 non-null   object
 10  customer_support_tickets  5000 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 429.8+ KB


In [6]:
data['monthly_revenue'] = data['monthly_revenue'].str.replace('$', '', regex=False).astype(float)

In [7]:
data['event_date'] = pd.to_datetime(data['event_date'] )

In [8]:
data.head()

,event_id,user_id,event_date,country,subscription_plan,device_type,traffic_source,event_type,session_duration_mins,monthly_revenue,customer_support_tickets
0,E50000,U10969,2024-01-18,UK,Free,Tablet,Referral,Login,37,56.27,2
1,E50001,U10772,2024-02-27,Australia,Basic,Mobile,Email,Feature_Use,142,81.08,1
2,E50002,U10048,2023-08-28,Australia,Free,Mobile,Ads,Feature_Use,176,304.46,4
3,E50003,U10368,2023-03-01,India,Basic,Desktop,Referral,Login,1,259.81,1
4,E50004,U11144,2023-08-10,Germany,Basic,Mobile,Organic,Login,39,119.56,0


In [9]:
data.duplicated().any()

np.False_

In [10]:
data['year'] = data['event_date'].dt.year


In [11]:
data['month'] = data['event_date'].dt.month


In [12]:
first_date = data.groupby('user_id').agg(first_date = ('event_date' , 'min'))

In [13]:
data = data.merge(first_date , how='left' , on='user_id')

In [14]:
data['cohurt_month'] = data['first_date'].dt.to_period("M")

In [15]:
#data['cohurt_month_name'] = data['first_date'].dt.month_name()

In [16]:
logins = data[data['event_type'] == 'Login']

In [17]:
logins = logins.groupby(by = ['user_id' , 'month']).agg(
    count_of_logins = ('event_type' , 'count')
).reset_index()

In [18]:
logins

,user_id,month,count_of_logins
0,U10000,3,1
1,U10000,8,1
2,U10001,2,1
3,U10001,4,1
4,U10001,6,1
...,...,...,...
2237,U11196,8,1
2238,U11197,4,1
2239,U11197,9,1
2240,U11197,11,1


In [19]:
data = data.merge(logins , how='left' , on=['user_id' ,'month'])

In [20]:
data['active_flag'] = data['count_of_logins'].map(
    lambda x: 0 if pd.isna(x) else 1
)

In [21]:
data.columns

Index(['event_id', 'user_id', 'event_date', 'country', 'subscription_plan',
       'device_type', 'traffic_source', 'event_type', 'session_duration_mins',
       'monthly_revenue', 'customer_support_tickets', 'year', 'month',
       'first_date', 'cohurt_month', 'count_of_logins', 'active_flag'],
      dtype='object')

In [22]:
data['event_type'].unique()

array(['Login', 'Feature_Use', 'Churn', 'Upgrade', 'Downgrade'],
      dtype=object)

In [23]:
data['traffic_source'].unique()

array(['Referral', 'Email', 'Ads', 'Organic'], dtype=object)

In [24]:
data['is_churn'] = (data['event_type'] == 'Churn').astype(int)

In [25]:
user_churn = data.groupby(['user_id', 'traffic_source'])['is_churn'].max().reset_index()

In [26]:
churn_rate = user_churn.groupby('traffic_source').agg(
    total_user = ('user_id' , 'nunique'),
    churned_users=('is_churn', 'sum')
).reset_index()
churn_rate['churn_rate'] = round(churn_rate['churned_users'] /churn_rate['total_user'] *100 , 2)

In [27]:
churn_rate

,traffic_source,total_user,churned_users,churn_rate
0,Ads,766,105,13.71
1,Email,805,111,13.79
2,Organic,785,142,18.09
3,Referral,765,114,14.90


In [28]:
data['is_upgrade'] = (data['event_type'] == 'Upgrade').astype(int)

In [29]:
user_upgrade = data.groupby(by = ['user_id' , 'traffic_source'])['is_upgrade'].max().reset_index()

In [30]:
user_upgrade_rate = user_upgrade.groupby('traffic_source').agg(
    tota_users = ('user_id' , 'nunique' ),
    upgraded_users = ('is_upgrade' , 'sum')
).reset_index()

user_upgrade_rate['upgrade_rate'] = round(user_upgrade_rate['upgraded_users']/user_upgrade_rate['tota_users'] * 100,2)

In [31]:
user_upgrade_rate

,traffic_source,tota_users,upgraded_users,upgrade_rate
0,Ads,766,129,16.84
1,Email,805,133,16.52
2,Organic,785,122,15.54
3,Referral,765,118,15.42


In [32]:
data.head()

,event_id,user_id,event_date,country,subscription_plan,device_type,traffic_source,event_type,session_duration_mins,monthly_revenue,customer_support_tickets,year,month,first_date,cohurt_month,count_of_logins,active_flag,is_churn,is_upgrade
0,E50000,U10969,2024-01-18,UK,Free,Tablet,Referral,Login,37,56.27,2,2024,1,2023-04-25,2023-04,2.0,1,0,0
1,E50001,U10772,2024-02-27,Australia,Basic,Mobile,Email,Feature_Use,142,81.08,1,2024,2,2023-09-08,2023-09,NaN,0,0,0
2,E50002,U10048,2023-08-28,Australia,Free,Mobile,Ads,Feature_Use,176,304.46,4,2023,8,2023-03-04,2023-03,NaN,0,0,0
3,E50003,U10368,2023-03-01,India,Basic,Desktop,Referral,Login,1,259.81,1,2023,3,2023-02-23,2023-02,1.0,1,0,0
4,E50004,U11144,2023-08-10,Germany,Basic,Mobile,Organic,Login,39,119.56,0,2023,8,2023-01-26,2023-01,1.0,1,0,0


In [33]:
user_data = data.groupby('user_id').agg(
    avg_session=('session_duration_mins', 'mean'),
    churn=('event_type', lambda x: (x == 'Churn').any()),
    upgrade=('event_type', lambda x: (x == 'Upgrade').any())
).reset_index()

In [34]:
user_data

,user_id,avg_session,churn,upgrade
0,U10000,131.333333,False,False
1,U10001,92.750000,False,False
2,U10002,135.250000,False,True
3,U10003,86.200000,True,False
4,U10004,80.333333,False,False
...,...,...,...,...
1173,U11195,79.400000,False,False
1174,U11196,80.333333,False,False
1175,U11197,60.000000,False,True
1176,U11198,87.000000,True,False


In [35]:
user_data.groupby('churn')['avg_session'].mean()

churn
False    89.177524
True     92.123275
Name: avg_session, dtype: float64

In [36]:
user_data.groupby('upgrade')['avg_session'].mean()

upgrade
False    90.242526
True     90.031363
Name: avg_session, dtype: float64

In [37]:
data.groupby('event_type')['session_duration_mins'].mean()

event_type
Churn          92.714286
Downgrade      92.887218
Feature_Use    91.431325
Login          89.141870
Upgrade        89.171053
Name: session_duration_mins, dtype: float64

In [38]:
data.head()

,event_id,user_id,event_date,country,subscription_plan,device_type,traffic_source,event_type,session_duration_mins,monthly_revenue,customer_support_tickets,year,month,first_date,cohurt_month,count_of_logins,active_flag,is_churn,is_upgrade
0,E50000,U10969,2024-01-18,UK,Free,Tablet,Referral,Login,37,56.27,2,2024,1,2023-04-25,2023-04,2.0,1,0,0
1,E50001,U10772,2024-02-27,Australia,Basic,Mobile,Email,Feature_Use,142,81.08,1,2024,2,2023-09-08,2023-09,NaN,0,0,0
2,E50002,U10048,2023-08-28,Australia,Free,Mobile,Ads,Feature_Use,176,304.46,4,2023,8,2023-03-04,2023-03,NaN,0,0,0
3,E50003,U10368,2023-03-01,India,Basic,Desktop,Referral,Login,1,259.81,1,2023,3,2023-02-23,2023-02,1.0,1,0,0
4,E50004,U11144,2023-08-10,Germany,Basic,Mobile,Organic,Login,39,119.56,0,2023,8,2023-01-26,2023-01,1.0,1,0,0


In [45]:
active_users= data[data['active_flag'] == 1]  

In [52]:
per_contry_revenue = active_users.groupby('country').agg(
    total_revenue  = ('monthly_revenue' , 'sum'),
    total_active_users = ('user_id' , 'nunique')
)
per_contry_revenue['rpm'] = round(per_contry_revenue['total_revenue']/per_contry_revenue['total_active_users'] ,2)

In [53]:
per_contry_revenue

,total_revenue,total_active_users,rpm
country,,,
Australia,125545.22,393,319.45
Canada,122333.40,402,304.31
Germany,116056.30,377,307.84
India,120931.13,396,305.38
UK,119517.77,382,312.87
USA,114834.47,390,294.45


In [54]:
active_users

,event_id,user_id,event_date,country,subscription_plan,device_type,traffic_source,event_type,session_duration_mins,monthly_revenue,customer_support_tickets,year,month,first_date,cohurt_month,count_of_logins,active_flag,is_churn,is_upgrade
0,E50000,U10969,2024-01-18,UK,Free,Tablet,Referral,Login,37,56.27,2,2024,1,2023-04-25,2023-04,2.0,1,0,0
3,E50003,U10368,2023-03-01,India,Basic,Desktop,Referral,Login,1,259.81,1,2023,3,2023-02-23,2023-02,1.0,1,0,0
4,E50004,U11144,2023-08-10,Germany,Basic,Mobile,Organic,Login,39,119.56,0,2023,8,2023-01-26,2023-01,1.0,1,0,0
5,E50005,U10700,2023-09-28,India,Pro,Tablet,Organic,Login,107,175.57,1,2023,9,2023-03-04,2023-03,1.0,1,0,0
7,E50007,U10511,2024-02-14,UK,Basic,Desktop,Email,Login,139,57.08,4,2024,2,2023-01-15,2023-01,1.0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4992,E54992,U10139,2023-03-05,Australia,Free,Desktop,Organic,Login,44,348.32,1,2023,3,2023-02-19,2023-02,1.0,1,0,0
4993,E54993,U10973,2023-03-27,India,Pro,Tablet,Organic,Login,2,466.57,2,2023,3,2023-03-27,2023-03,1.0,1,0,0
4996,E54996,U10422,2023-10-01,Germany,Basic,Tablet,Organic,Login,129,412.90,1,2023,10,2023-01-05,2023-01,2.0,1,0,0
4998,E54998,U11122,2024-04-08,Germany,Basic,Tablet,Ads,Login,126,191.87,3,2024,4,2023-01-02,2023-01,2.0,1,0,0


In [60]:
tiekt_ratio = active_users.groupby('country').agg(
    total_ticket_per_cntry = ('customer_support_tickets' , 'sum')
)

tiekt_ratio['tckt_ratio'] = round((tiekt_ratio['total_ticket_per_cntry'] / active_users['customer_support_tickets'].sum())*100,2)

In [61]:
tiekt_ratio

,total_ticket_per_cntry,tckt_ratio
country,,
Australia,742,17.26
Canada,748,17.40
Germany,678,15.77
India,720,16.74
UK,690,16.05
USA,722,16.79
